### 다섯 번째(최종)
### 앞서 세 번째와 네 번째에서 만든 csv파일로 통합합니다.
#### 이때, 데이터프레임의 통합 기준은 ticker입니다.

In [88]:
import pandas as pd
import numpy as np

In [89]:
api_info = pd.read_csv('0129rs&p500_add_info.csv')
crawling_info = pd.read_csv('0130snp500.csv')

In [90]:
api_info

,Unnamed: 0,forwardPE,marketCap
0,MMM,17.684261,106461995008
1,ABT,25.496847,215102455808
2,ABBV,8.573071,184283545600
3,ABMD,70.903350,16244866048
4,ACN,27.741436,166342803456
...,...,...,...
500,YUM,25.768750,31094427648
501,ZBRA,27.590687,21168209920
502,ZBH,20.064720,32772724736
503,ZION,10.946078,7328259584


In [91]:
# ticker 기준으로 합칠 것이기에 컬럼명을 바꿔줍니다.
api_info.columns = ['ticker','forwardPE','marketCap']

In [92]:
crawling_info

,Unnamed: 0,level_0,index,ticker,ann_growth_rate,total_rev_ttm,num_of_year,recent_year
0,0,0,0,MMM,0.55,32184000,4,2020
1,1,1,1,ABT,8.11,34608000,4,2020
2,2,2,2,ABBV,8.58,34057000,3,2019
3,3,3,3,ABMD,23.60,802857,4,2020
4,4,4,4,ACN,6.43,44730266,4,2020
...,...,...,...,...,...,...,...,...
499,499,499,49,XYL,5.60,5249000,3,2019
500,500,500,0,YUM,-2.42,5603000,3,2019
501,501,501,1,ZBRA,9.77,4332000,3,2019
502,502,502,2,ZBH,1.01,7790500,3,2019


In [93]:
# Unnamed: 0, level_0, index은 필요없는 컬럼이기에 제거합니다.
crawling_info.drop(['Unnamed: 0','level_0','index'], axis='columns', inplace=True)

In [94]:
# ticker를 중심으로 합칩니다.
merge_data = pd.merge(crawling_info,api_info, how='left',on='ticker')

In [95]:
merge_data

,ticker,ann_growth_rate,total_rev_ttm,num_of_year,recent_year,forwardPE,marketCap
0,MMM,0.55,32184000,4,2020,17.684261,106461995008
1,ABT,8.11,34608000,4,2020,25.496847,215102455808
2,ABBV,8.58,34057000,3,2019,8.573071,184283545600
3,ABMD,23.60,802857,4,2020,70.903350,16244866048
4,ACN,6.43,44730266,4,2020,27.741436,166342803456
...,...,...,...,...,...,...,...
499,XYL,5.60,5249000,3,2019,37.605660,17961019392
500,YUM,-2.42,5603000,3,2019,25.768750,31094427648
501,ZBRA,9.77,4332000,3,2019,27.590687,21168209920
502,ZBH,1.01,7790500,3,2019,20.064720,32772724736


In [96]:
# total_rev_ttm는 기본 1000단위이기에 marketCap과의 단위를 맞추기 위해 1000을 곱해줍니다.
merge_data.total_rev_ttm = merge_data.total_rev_ttm * 1000
merge_data

,ticker,ann_growth_rate,total_rev_ttm,num_of_year,recent_year,forwardPE,marketCap
0,MMM,0.55,32184000000,4,2020,17.684261,106461995008
1,ABT,8.11,34608000000,4,2020,25.496847,215102455808
2,ABBV,8.58,34057000000,3,2019,8.573071,184283545600
3,ABMD,23.60,802857000,4,2020,70.903350,16244866048
4,ACN,6.43,44730266000,4,2020,27.741436,166342803456
...,...,...,...,...,...,...,...
499,XYL,5.60,5249000000,3,2019,37.605660,17961019392
500,YUM,-2.42,5603000000,3,2019,25.768750,31094427648
501,ZBRA,9.77,4332000000,3,2019,27.590687,21168209920
502,ZBH,1.01,7790500000,3,2019,20.064720,32772724736


In [97]:
# PSR(주가매출비율, 제가 좋아하는 지표)를 추가해줍니다.
merge_data['PSR'] = merge_data['marketCap'] / merge_data['total_rev_ttm']
merge_data.PSR

0       3.307917
1       6.215397
2       5.411033
3      20.233823
4       3.718798
         ...    
499     3.421798
500     5.549603
501     4.886475
502     4.206755
503     2.627558
Name: PSR, Length: 504, dtype: float64

In [98]:
# forwardPE과 PSR의 소숫점이 많으니 반올림 해줍니다.
merge_data = merge_data.round({'forwardPE': 2, 'PSR': 2})
merge_data

,ticker,ann_growth_rate,total_rev_ttm,num_of_year,recent_year,forwardPE,marketCap,PSR
0,MMM,0.55,32184000000,4,2020,17.68,106461995008,3.31
1,ABT,8.11,34608000000,4,2020,25.50,215102455808,6.22
2,ABBV,8.58,34057000000,3,2019,8.57,184283545600,5.41
3,ABMD,23.60,802857000,4,2020,70.90,16244866048,20.23
4,ACN,6.43,44730266000,4,2020,27.74,166342803456,3.72
...,...,...,...,...,...,...,...,...
499,XYL,5.60,5249000000,3,2019,37.61,17961019392,3.42
500,YUM,-2.42,5603000000,3,2019,25.77,31094427648,5.55
501,ZBRA,9.77,4332000000,3,2019,27.59,21168209920,4.89
502,ZBH,1.01,7790500000,3,2019,20.06,32772724736,4.21


In [99]:
# 이제 파일을 저장합니다.
merge_data.to_csv('0130snp500_data.csv')